Note: To display the density grid calculations, in VisIt, load in the .cube file (must select file type and deselect default option), and deselect options to apply options to all plots/windows. When making density grid plots, use the minimum and maximum energy values displayed inthe legend (don't turn off annotations in VisIt immediately) to *normalize* the color gradient. Make a low quality plot and find a good viewing angle; then, increase the resolution and print screen as png. 

TODO: try making density grid plots using alternative software posted in slack.

In [1]:
using PorousMaterials
using JLD2

In [2]:
@eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(),"structural_relaxation","post-relaxation_cifs")

"/home/ng/DTRA/structural_relaxation/post-relaxation_cifs"

In [3]:
params = Dict(:adsorbate   => "Xe",
              :ljff        => "UFF",
              :grid_points => (100, 100, 100), # (100, 100, 100)
              :repfactors  => (1,1,1),
              :temperature => 298.0, 
              :n_rotations => 750,
              :energy_min  => Tuple{Float64,CartesianIndex{3}},
              :xyz_file    => true,
              :vtk_file    => true)

Dict{Symbol,Any} with 9 entries:
  :ljff        => "UFF"
  :adsorbate   => "Xe"
  :grid_points => (100, 100, 100)
  :temperature => 298.0
  :energy_min  => Tuple{Float64,CartesianIndex{3}}
  :xyz_file    => true
  :vtk_file    => true
  :repfactors  => (1, 1, 1)
  :n_rotations => 750

In [4]:
# I need to put place the adsorbate at the location of the minimum energy
function adsorbate_in_mof(energy_grid_params::Dict{String,Dict{Symbol,Any}}, xtal::Crystal, grid::Grid)
    filename = energy_grid_params[xtal.name][:adsorbate] * "_in_" * xtal.name
    
    molecule = Molecule(energy_grid_params[xtal.name][:adsorbate])
    xyz = Tuple([energy_grid_params[xtal.name][:energy_min][2][i] for i in 1:3])
    
    xf_minE = id_to_xf(xyz, grid.n_pts) # fractional coords of min
    x_minE  = Cart(Frac(xf_minE), xtal.box) # Cartesian coords of min
    # make struct
    xe_atom_c = Atoms(1, [molecule.species], x_minE) 

    return write_xyz(xe_atom_c, filename)
end

adsorbate_in_mof (generic function with 1 method)

In [5]:
# min_E, argmin_E = findmin(grid.data)
# xyz = Tuple([argmin_E[i] for i in 1:3])
# xf_minE = id_to_xf(xyz, grid.n_pts)
# x_minE  = Cart(Frac(xf_minE), crystal.box) 
# xe_atom_c = Atoms(1, [:Xe], x_minE) 
# write_xyz(xe_atom_c, "blah.xyz")

In [6]:
xtal_names = ["NiPyC2_experiment.cif", 
              "NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif"]

energy_grid_params = Dict(zip(xtal_names, [params for i in 1:length(xtal_names)]))

Dict{String,Dict{Symbol,Any}} with 2 entries:
  "NiPyC2_experiment.cif"   => Dict{Symbol,Any}(:ljff=>"UFF",:adsorbate=>"Xe",:…
  "NiPyC2_relax_sc211_meta… => Dict{Symbol,Any}(:ljff=>"UFF",:adsorbate=>"Xe",:…

In [7]:
# for xtal in xtal_names
#     # load crystal and replicat (if needed)
#     crystal = replicate(Crystal(xtal), energy_grid_params[xtal][:repfactors])
#     strip_numbers_from_atom_labels!(crystal)
    
#     # assign energy calculation parameters
#     mol    = Molecule(energy_grid_params[xtal][:adsorbate])
#     ljff   = LJForceField(energy_grid_params[xtal][:ljff])
#     points = energy_grid_params[xtal][:grid_points]
#     temp   = energy_grid_params[xtal][:temperature]
#     rot    = energy_grid_params[xtal][:n_rotations]
    
#     # perform energy grid calculation
#     grid = energy_grid(crystal, mol, ljff; n_pts=points, temperature=temp, n_rotations=rot)
    
#     # find the value and location of the minimum energy 
#     energy_grid_params[xtal][:energy_min] = findmin(grid.data)
#     println(xtal, " energy min = ", energy_grid_params[xtal][:energy_min])
    
#     # write cube file using crystal name for filename
#     write_cube(grid, crystal.name)
    
#     # write xyz file
#     if energy_grid_params[xtal][:xyz_file]
#         atoms_c = Cart(crystal.atoms, crystal.box)
#         write_xyz(crystal)
#     end
    
#     # write vtk file
#     if energy_grid_params[xtal][:vtk_file]
#         write_vtk(crystal)
#     end
    
#     adsorbate_in_mof(energy_grid_params, crystal, grid)
# end

In [8]:
for xtal in xtal_names
    # load crystal and replicate 
    crystal = Crystal(xtal)
    energy_grid_params[xtal][:repfactors] = replication_factors(crystal, 14.0)
    crystal = replicate(crystal, energy_grid_params[xtal][:repfactors])
    strip_numbers_from_atom_labels!(crystal)
    
    
    
    # assign energy calculation parameters
    mol    = Molecule(energy_grid_params[xtal][:adsorbate])
    ljff   = LJForceField(energy_grid_params[xtal][:ljff])
    points = energy_grid_params[xtal][:grid_points]
    temp   = energy_grid_params[xtal][:temperature]
    rot    = energy_grid_params[xtal][:n_rotations]
    
    pmin      = -2 # minimum pessure in log10, units: bar
    pmax      = 1.1 # maximum pressure (actual value), units: bar
    nstep     = 15 # number of intervals in pressure range
    pressures = 10 .^ range(pmin, stop=log10(pmax), length=nstep)
    nburn     = 50000
    nsample   = 50000
    
    # loop over pressures
    for p in 1:length(pressures)
        filename = μVT_output_filename(crystal, mol, temp, pressures[p], ljff, nburn, nsample)
        println(filename)

        @load joinpath(PorousMaterials.PATH_TO_SIMS, filename) results

        grid = results["density grid"]

        # find the value and location of the minimum energy 
        energy_grid_params[xtal][:energy_min] = findmin(grid.data)
        println(xtal, " energy min = ", energy_grid_params[xtal][:energy_min])

        # write cube file using crystal name for filename
        write_cube(grid, filename)
    end
    
    # write xyz file
    if energy_grid_params[xtal][:xyz_file]
        atoms_c = Cart(crystal.atoms, crystal.box)
        write_xyz(crystal)
    end
    
    # write vtk file
    if energy_grid_params[xtal][:vtk_file]
        write_vtk(crystal)
    end
    
#     adsorbate_in_mof(energy_grid_params, crystal, grid)
end

┌ Info: Crystal NiPyC2_experiment.cif has Pn space group. I am converting it to P1 symmetry.
│         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ PorousMaterials /home/ng/.julia/dev/PorousMaterials/src/crystal.jl:426


muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0100000bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids/muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0100000bar_UFF_50000burn_50000sample.jld2.cube
muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0139899bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids/muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0139899bar_UFF_50000burn_50000sample.jld2.cube
muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0195717bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids/muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0195717bar_UFF_50000burn_50000sample.jld2.cube
muVT_Xe_in_NiPyC2_experiment.cif_298.000K_0.0273805bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids

NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids/muVT_Xe_in_NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif_298.000K_0.4017458bar_UFF_50000burn_50000sample.jld2.cube
muVT_Xe_in_NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif_298.000K_0.5620373bar_UFF_50000burn_50000sample.jld2
NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids/muVT_Xe_in_NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif_298.000K_0.5620373bar_UFF_50000burn_50000sample.jld2.cube
muVT_Xe_in_NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif_298.000K_0.7862831bar_UFF_50000burn_50000sample.jld2
NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/data/grids/muVT_Xe_in_NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif_298.000K_0.7862831bar